# [75.06 / 95.58] Organización de Datos <br> Trabajo Práctico 2: Machine Learning

**Grupo 30: Datatouille**

- 101055 - Bojman, Camila
- 100029 - del Mazo, Federico
- 100687 - Hortas, Cecilia
- 97649 - Souto, Rodrigo

**https://github.com/FdelMazo/7506-Datos**

**https://www.kaggle.com/datatouille2018/competitions**

In [ ]:
# https://github.com/Kaggle/kaggle-api
# Ver la documentación de la API de kaggle para saber como usar los tokens
# Antes de comenzar, buscar las credenciales de kaggle (el nombre de usuario, y el token) y setearlos en las siguientes dos lineas

#%env KAGGLE_USERNAME="xxx"
#%env KAGGLE_KEY="xxx"

#!pip install kaggle
#!kaggle competitions download -c trocafone -p data
#!unzip -q data/events_up_to_01062018.csv.zip -d data
#!rm data/events_up_to_01062018.csv.zip
#!ls data/

### Lista de robos

* [ ] https://github.com/urielkelman/abracadata/tree/master/TP2

* [ ] https://github.com/GastonMontes/Datos-TP2

* [ ] https://github.com/MatiasReimondo/Datos

* [ ] Ver diapos argerich feature engineering

* [ ] Spammear ramos mejia de preguntas

### Lista de cosas a hacer

* [ ] Dividir en set de entrenamientos y set de test. Ojo, hay que hacerlo con tiempo! No se puede hacer al azar. Los primeros meses entrenan a los siguientes. Etc

* [ ] Identificar bias y varianza, ploteando error de set de entrenamiento y error de set de test en funcion de cantidad de datos en set de entrenamiento (mismo plot)

* [ ] Perturbar datos de entrada -> reducir overfitting

* [ ] Feature engineering --> no olvidar documentar y versionar!!!

* [ ] Catboost

* [ ] Feature: dia de la semana

* [ ] Loopear interacciones importantes! Random forest claveee

* [ ] No perder nombre de feature original

* [ ] Scrapear trocafone --> Precios

* [ ] Xgboost no sirve para importancia de features. No es estable
